In [3]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag

sentences = pd.read_csv("reviews.csv")
data = [pos_tag(word_tokenize(sentence.lower())) for sentence in sentences['reviews']]
data

[[('it', 'PRP'),
  ('is', 'VBZ'),
  ('very', 'RB'),
  ('gud', 'JJ'),
  ('place', 'NN'),
  ('to', 'TO'),
  ('get', 'VB'),
  ('food', 'NN'),
  ('the', 'DT'),
  ('food', 'NN'),
  ('is', 'VBZ'),
  ('too', 'RB'),
  ('gud', 'JJ'),
  ('and', 'CC'),
  ('mouth', 'JJ'),
  ('water', 'NN'),
  ('fascinating', 'VBG'),
  ('quality', 'NN'),
  ('and', 'CC'),
  ('quantity', 'NN'),
  ('that', 'IN'),
  ('u', 'JJ'),
  ('ever', 'RB'),
  ('going', 'VBG'),
  ('to', 'TO'),
  ('get', 'VB'),
  ('awesome', 'JJ'),
  ('food', 'NN'),
  ('really', 'RB'),
  ('loved', 'VBD'),
  ('it', 'PRP')],
 [('great', 'JJ'),
  ('food', 'NN'),
  (',', ','),
  ('indoor', 'NN'),
  ('and', 'CC'),
  ('outdoor', 'NN'),
  ('seating', 'NN')],
 [('loved', 'VBN'),
  ('all', 'DT'),
  ('our', 'PRP$'),
  ('food', 'NN'),
  (',', ','),
  ('licensed', 'VBD'),
  (',', ','),
  ('fast', 'RB'),
  (',', ','),
  ('efficient', 'JJ'),
  ('service', 'NN')],
 [('manager', 'NN'),
  ('very', 'RB'),
  ('friendly', 'RB'),
  (',', ','),
  ('helpful', 'JJ'),
  ('

In [75]:
# from gensim.models import Word2Vec
# from gensim.test.utils import common_texts

# model = Word2Vec(data, size=855, window=5, min_count=1, workers=4)
# model.train(data, total_examples=855, epochs=3)

(24407, 33501)

In [77]:
# aspects = ["food", "place", "service", "price"]

# for aspect in aspects:
#     print(aspect + ": " + str(model.wv.similarity("owner", aspect)))

food: 0.9992700116871447
place: 0.999254615754284
service: 0.9992557473736113
price: 0.9991627496600437


In [4]:
from nltk.corpus import wordnet as wn

def nounify(word, pos):
    if pos == 'n':
        return word
    synsets = wn.synsets(word, pos=pos)
    lemmas = [l for s in synsets for l in s.lemmas() if s.pos() == pos]
    related = [(l, l.derivationally_related_forms()) for l in lemmas]
    related_noun = [l for r in related for l in r[1] if l.synset().pos() == 'n']
    words = [l.name() for l in related_noun]
    
    ln = len(words)
    result = ""
    prob = 0
    for w in set(words):
        if float(words.count(w))/ln > prob:
            result = w
            prob = float(words.count(w))/ln
            
    return result

def wu_palmer_similarity(s1, s2):
    hp1, hp2 = s1.hypernym_paths()[0], s2.hypernym_paths()[0]
    d1, d2 = len(hp1), len(hp2)

    d = min(d1, d2)
    while (hp1[d-1]!=hp2[d-1] and d > 0):
            d -= 1
    
    return float(2 * d) / (d1 + d2)

In [5]:
def wordnet_pos_code(tag):
    if tag.startswith('NN'):
        return wn.NOUN
    elif tag.startswith('VB'):
        return wn.VERB
    elif tag.startswith('JJ'):
        return wn.ADJ
    elif tag.startswith('RB'):
        return wn.ADV
    else:
        return ''

In [16]:
aspects = ["food", "price", "service", "place"]
cnt = 0

for sent in data:
    string = ""
    for word in sent:
        string += word[0] + ' '
        test = nounify(word[0], wordnet_pos_code(word[1]))
        
        for aspect in aspects:
            sa = wn.synsets(aspect)
            st = wn.synsets(test)
            maks = 0
            for x in sa:
                for y in st:
                    if x.pos() == y.pos() and x.pos() == 'n':
                        maks = max(maks, wu_palmer_similarity(x, y))
            if (maks > 0.8):
                print(aspect + '\t' + word[0] + '\t' + str(maks))
    print(string)
    cnt += 1
    if (cnt==10):
        break

service	place	0.8235294117647058
place	place	1.0
food	food	1.0
food	food	1.0
food	water	0.9090909090909091
place	quality	0.8571428571428571
food	food	1.0
it is very gud place to get food the food is too gud and mouth water fascinating quality and quantity that u ever going to get awesome food really loved it 
food	food	1.0
service	seating	0.9411764705882353
great food , indoor and outdoor seating 
food	food	1.0
service	service	1.0
place	service	0.8235294117647058
loved all our food , licensed , fast , efficient service 
service	helpful	0.9230769230769231
manager very friendly , helpful and courteous 
try it when your in singapore , you wont be disappointed 
we will be back 
food	food	1.0
food	culture	0.8333333333333334
ok so we searched for best india restaurants in singapore after so much struggle finding good india food and got curry culture from tripadvisor 
service	located	0.8571428571428571
food	food	1.0
its located in a very amazing food complex on the cuppage road 
service	staff